In [1]:
import pandas as pd
import numpy as np
from glob import glob

from PIL import Image
import cv2
from tqdm import tqdm


import os
import json

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter('ignore')
import matplotlib.pyplot as plt


In [2]:
data_path = ''

train_path = data_path +'./train'

test_path = data_path +'./test'

new_image_directory = data_path + './new_images'

new_train_image_directory = new_image_directory +'/train'

new_test_image_directory = new_image_directory +'/test'

In [3]:
def make_new_dir(path):
    if os.path.isdir(path) == False:
        os.makedirs(path)

In [4]:
make_new_dir(new_image_directory)
make_new_dir(new_train_image_directory)
make_new_dir(new_test_image_directory)

In [5]:
target = r"C:/AIFactory/ETC/교통수신호/json/*"
json_path = np.array(sorted(glob(target), key = lambda x: int(x.split('/')[-1].split('_')[-1].split('.')[0])))
json_path[:5]

array(['C:/AIFactory/ETC/교통수신호/json\\file_0.json',
       'C:/AIFactory/ETC/교통수신호/json\\file_1.json',
       'C:/AIFactory/ETC/교통수신호/json\\file_2.json',
       'C:/AIFactory/ETC/교통수신호/json\\file_3.json',
       'C:/AIFactory/ETC/교통수신호/json\\file_4.json'], dtype='<U41')

In [6]:
action_information = pd.read_csv('action_information.csv')

In [7]:
action_information

,action,Label
0,우측에서 좌측으로,0
1,전방정지,1
2,전후방 동시정지,2
3,좌우측방 동시정지,3
4,좌측에서 우측으로,4
5,후방정지,5


In [8]:
classes = pd.get_dummies(action_information[['Label']], columns = ['Label']).to_numpy()
classes

array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]], dtype=uint8)

In [9]:
train_directories = np.array(sorted(glob(train_path + '/**'), key = lambda x: int(x.split('/')[-1].split('_')[-1])))

In [11]:
for i, train_directory in tqdm(enumerate(train_directories)) : 
    file_name = train_directory.split('/')[-1].split('\\')[-1]
    make_new_dir(new_train_image_directory + '/'+file_name)
    
    image_paths = sorted(glob(train_directory + '/*.jpg'), key = lambda x: int(x.split('\\')[-1].split('.')[0]))
    
    target = r"C:/AIFactory/ETC/교통수신호/json/*"
    json_path = np.array(sorted(glob(target), key = lambda x: int(x.split('/')[-1].split('_')[-1].split('.')[0])))
    js_path= json_path[i]
    js = json.load(open(js_path))
    
    target = js.get('action')
    target = classes[target]
    bounding_boxes = js.get('sequence').get('bounding_box')
    bounding_boxes = [(float(a),float(b),float(c),float(d)) for a,b,c,d in bounding_boxes] 

    for image_path, bounding_box in zip(image_paths, bounding_boxes) : 
        image = Image.open(image_path)
        image = image.crop(bounding_box)
        image = image.resize((224,224))
        image.save(new_train_image_directory + image_path.split('/train')[1])       

142it [15:32,  6.57s/it]


In [13]:
test_directories = np.array(sorted(glob(test_path + '/**'), key = lambda x: int(x.split('/')[-1].split('_')[-1])))

In [14]:
for i, test_directory in tqdm(enumerate(test_directories)) : 
    file_name = test_directory.split('/')[-1].split('\\')[-1]
    make_new_dir(new_test_image_directory + '/'+file_name)
    
    image_paths = sorted(glob(test_directory + '/*.jpg'), key = lambda x: int(x.split('\\')[-1].split('.')[0]))
    
    target = r"C:/AIFactory/ETC/교통수신호/json1/*"
    json_path = np.array(sorted(glob(target), key = lambda x: int(x.split('/')[-1].split('_')[-1].split('.')[0])))
    js_path= json_path[i]
    js = json.load(open(js_path))
    
    target = js.get('action')
    target = classes[target]
    bounding_boxes = js.get('sequence').get('bounding_box')
    bounding_boxes = [(float(a),float(b),float(c),float(d)) for a,b,c,d in bounding_boxes] 

    for image_path, bounding_box in zip(image_paths, bounding_boxes) : 
        image = Image.open(image_path)
        image = image.crop(bounding_box)
        image = image.resize((224,224))
        image.save(new_test_image_directory + image_path.split('/test')[1])   

45it [03:58,  5.29s/it]
